In [40]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, LSTM, Conv2D, MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam, SGD
import pandas as pd
import numpy as np

In [58]:
dataset_path = '../datasets/synergy-final-iter1-features/'

device = 'xdk_1'

df = pd.read_pickle(dataset_path + device + '.p')
df_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# filter activities
use_activities = [0,1,2,3,4,5,6,7]
num_classes = len(use_activities)
df_labels = df_labels.loc[df_labels.label.isin(use_activities)]
df = df.loc[df.index.isin(df_labels.index)]

df = df.filter(regex='accel_')

# X = df.values
# y = df_labels['label'].values

labels = df_labels[['label']]

train_test_split = np.random.rand(len(df)) < 0.70
x_train = df[train_test_split]
y_train = labels[train_test_split]
x_test = df[~train_test_split]
y_test = labels[~train_test_split]

x_train = x_train.values
x_test = x_test.values

y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

In [59]:
timesteps = 3
data_dim = round(len(x_train[0]) / timesteps)
batch_size = 64

In [60]:
x_train = x_train.reshape(len(x_train), timesteps, data_dim)
x_test = x_test.reshape(len(x_test), timesteps, data_dim)

In [61]:
import math

x_train = x_train[:math.floor(len(x_train) / batch_size) * batch_size]
x_test = x_test[:math.floor(len(x_test) / batch_size) * batch_size]
y_train = y_train[:math.floor(len(y_train) / batch_size) * batch_size]
y_test = y_test[:math.floor(len(y_test) / batch_size) * batch_size]

In [63]:
lstm_output_size = 64

model = Sequential()
model.add(LSTM(lstm_output_size, return_sequences=True, stateful=True,
               batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(LSTM(lstm_output_size, return_sequences=True, stateful=True))
model.add(LSTM(lstm_output_size, stateful=True))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=100)
score = model.evaluate(x_test, y_test, batch_size=batch_size)
score

Epoch 1/100
384/384 [==============================] - 1s - loss: 2.0773 - acc: 0.1328     
Epoch 2/100
384/384 [==============================] - 0s - loss: 2.0773 - acc: 0.1224     
Epoch 3/100
384/384 [==============================] - 0s - loss: 2.0747 - acc: 0.1146     
Epoch 4/100
384/384 [==============================] - 0s - loss: 2.0466 - acc: 0.1510     
Epoch 5/100
384/384 [==============================] - 0s - loss: 2.0498 - acc: 0.1562     
Epoch 6/100
384/384 [==============================] - 0s - loss: 2.0367 - acc: 0.1771     
Epoch 7/100
384/384 [==============================] - 0s - loss: 2.0456 - acc: 0.1510     
Epoch 8/100
384/384 [==============================] - 0s - loss: 2.0403 - acc: 0.1589     
Epoch 9/100
384/384 [==============================] - 0s - loss: 2.0302 - acc: 0.1536     
Epoch 10/100
384/384 [==============================] - 0s - loss: 2.0167 - acc: 0.1693     
Epoch 11/100
384/384 [==============================] - 0s - loss: 2.0180 - acc

384/384 [==============================] - 0s - loss: 1.7164 - acc: 0.2786     
Epoch 90/100
384/384 [==============================] - 0s - loss: 1.7031 - acc: 0.2969     
Epoch 91/100
384/384 [==============================] - 0s - loss: 1.6816 - acc: 0.3151     
Epoch 92/100
384/384 [==============================] - 0s - loss: 1.6741 - acc: 0.2760     
Epoch 93/100
384/384 [==============================] - 0s - loss: 1.7178 - acc: 0.3203     
Epoch 94/100
384/384 [==============================] - 0s - loss: 1.7219 - acc: 0.2865     
Epoch 95/100
384/384 [==============================] - 0s - loss: 1.7296 - acc: 0.3047     
Epoch 96/100
384/384 [==============================] - 0s - loss: 1.6852 - acc: 0.2917     
Epoch 97/100
384/384 [==============================] - 0s - loss: 1.6442 - acc: 0.3203     
Epoch 98/100
384/384 [==============================] - 0s - loss: 1.6773 - acc: 0.3307     
Epoch 99/100
384/384 [==============================] - 0s - loss: 1.6813 - acc: 0.

[1.6782991886138916, 0.3515625]

In [44]:
filters = 250
kernel_size = 3
epochs = 100

model = Sequential()
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 input_shape=(timesteps, data_dim),
                 strides=1))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=epochs,
          batch_size=batch_size)
score = model.evaluate(x_test, y_test, batch_size=batch_size)
score

Epoch 1/100
384/384 [==============================] - 0s - loss: 2.2155 - acc: 0.1510     
Epoch 2/100
384/384 [==============================] - 0s - loss: 2.1785 - acc: 0.1536     
Epoch 3/100
384/384 [==============================] - 0s - loss: 2.1904 - acc: 0.1328     
Epoch 4/100
384/384 [==============================] - 0s - loss: 2.1649 - acc: 0.1120     
Epoch 5/100
384/384 [==============================] - 0s - loss: 2.1615 - acc: 0.1094     
Epoch 6/100
384/384 [==============================] - 0s - loss: 2.1331 - acc: 0.1354     
Epoch 7/100
384/384 [==============================] - 0s - loss: 2.1284 - acc: 0.1328     
Epoch 8/100
384/384 [==============================] - 0s - loss: 2.1546 - acc: 0.1432     
Epoch 9/100
384/384 [==============================] - 0s - loss: 2.1456 - acc: 0.1224     
Epoch 10/100
384/384 [==============================] - 0s - loss: 2.0749 - acc: 0.1641     
Epoch 11/100
384/384 [==============================] - 0s - loss: 2.1184 - acc

384/384 [==============================] - 0s - loss: 2.0642 - acc: 0.1224     
Epoch 90/100
384/384 [==============================] - 0s - loss: 2.0358 - acc: 0.1927     
Epoch 91/100
384/384 [==============================] - 0s - loss: 2.0535 - acc: 0.1484     
Epoch 92/100
384/384 [==============================] - 0s - loss: 2.0443 - acc: 0.1875     
Epoch 93/100
384/384 [==============================] - 0s - loss: 2.0514 - acc: 0.1562     
Epoch 94/100
384/384 [==============================] - 0s - loss: 2.0501 - acc: 0.1797     
Epoch 95/100
384/384 [==============================] - 0s - loss: 2.0407 - acc: 0.1510     
Epoch 96/100
384/384 [==============================] - 0s - loss: 2.0594 - acc: 0.1510     
Epoch 97/100
384/384 [==============================] - 0s - loss: 2.0609 - acc: 0.1458     
Epoch 98/100
384/384 [==============================] - 0s - loss: 2.0515 - acc: 0.1589     
Epoch 99/100
384/384 [==============================] - 0s - loss: 2.0511 - acc: 0.

[2.0652642488479613, 0.13125000000000001]

In [8]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

NameError: name 'pydot' is not defined